---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa',
          'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National',
          'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 
          'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee',
          'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho',
          'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii',
          'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 
          'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico',
          'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 
          'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri',
          'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 
          'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska',
          'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 
          'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico',
          'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands',
          'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 
          'ND': 'North Dakota', 'VA': 'Virginia'}

In [11]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    state_towns_list  = []
    
    with open("university_towns.txt") as file:
        for line in file:
            line = line[:-1]
            if line[-6:] == '[edit]':
                state = line[:-6]
            elif '(' in line:
                town = line[:line.index('(')-1]
                state_towns_list.append([state, town])
            else:
                town = line
                state_towns_list.append([state, town])
    
    state_towns_df = pd.DataFrame(state_towns_list, columns=['State','RegionName'])
    return state_towns_df
print(get_list_of_university_towns())

             State                    RegionName
0          Alabama                        Auburn
1          Alabama                      Florence
2          Alabama                  Jacksonville
3          Alabama                    Livingston
4          Alabama                    Montevallo
5          Alabama                          Troy
6          Alabama                    Tuscaloosa
7          Alabama                      Tuskegee
8           Alaska                     Fairbanks
9          Arizona                     Flagstaff
10         Arizona                         Tempe
11         Arizona                        Tucson
12        Arkansas                   Arkadelphia
13        Arkansas                        Conway
14        Arkansas                  Fayetteville
15        Arkansas                     Jonesboro
16        Arkansas                      Magnolia
17        Arkansas                    Monticello
18        Arkansas                  Russellville
19        Arkansas  

In [12]:
def get_gdp():
    GDP_lev = pd.read_excel('gdplev.xls',usecols=['Unnamed: 4', 'Unnamed: 6'], skiprows=7,)
    GDP_lev = GDP_lev.rename(columns={'Unnamed: 4':'Quarters', 'Unnamed: 6': 'GDP'})
    return GDP_lev
get_gdp()

,Quarters,GDP
0,1947q1,1934.5
1,1947q2,1932.3
2,1947q3,1930.3
3,1947q4,1960.7
4,1948q1,1989.5
5,1948q2,2021.9
6,1948q3,2033.2
7,1948q4,2035.3
8,1949q1,2007.5
9,1949q2,2000.8


In [13]:
def get_recession_start():
    
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    GDP_lev = get_gdp()
    for x in range(0,GDP_lev.shape[0] - 2):
        if (GDP_lev['GDP'][x] > GDP_lev['GDP'][x+1]) & (GDP_lev['GDP'][x+1] > GDP_lev['GDP'][x+2]):
            start = GDP_lev.iloc[x-1][0]
    return start
print(get_recession_start())    

2008q3


In [14]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    GDP_lev = get_gdp()
    start = get_recession_start()
    l = []
    for x in range(GDP_lev.index[GDP_lev['Quarters'] == start][0], GDP_lev.shape[0] - 2):
        if (GDP_lev['GDP'][x] < GDP_lev['GDP'][x+1]) & (GDP_lev['GDP'][x+1] < GDP_lev['GDP'][x+2]):
            l.append(GDP_lev['Quarters'][x+2])
    end = l[0]
    return end
print(get_recession_end())

2009q4


In [15]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    GDP_lev = get_gdp()
    start = get_recession_start()
    end = get_recession_end()
    l = []
    for x in range(GDP_lev.index[GDP_lev['Quarters'] == start][0], GDP_lev.index[GDP_lev['Quarters'] == end][0]):
        if (GDP_lev['GDP'][x] < GDP_lev['GDP'][x+1]) & (GDP_lev['GDP'][x+1] < GDP_lev['GDP'][x+2]):
            l.append(GDP_lev['Quarters'][x+2])
    min_value = 10000000
    for x in range(GDP_lev.index[GDP_lev['Quarters'] == start][0], GDP_lev.index[GDP_lev['Quarters'] == end][0]):
        if GDP_lev['GDP'][x] < min_value:
            min_value = GDP_lev['GDP'][x]
            bottom = GDP_lev['Quarters'][x] 
    return bottom
print(get_recession_bottom())

2009q2


In [16]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    housing_data = pd.read_csv("City_Zhvi_AllHomes.csv")
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa',
          'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National',
          'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 
          'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee',
          'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho',
          'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii',
          'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 
          'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico',
          'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 
          'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri',
          'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 
          'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska',
          'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 
          'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico',
          'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands',
          'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 
          'ND': 'North Dakota', 'VA': 'Virginia'}
    housing_data['State'].replace(states, inplace=True)
    housing_data = housing_data.set_index(['State', 'RegionName'])
    housing_data = housing_data.iloc[:,49:250]
    
    def name_quarters(column):
        if column.endswith(('01', '02', '03')):
            name = column[:4]+'q1'
        elif column.endswith(('04', '05', '06')):
            name = column[:4]+'q2'
        elif column.endswith(('07', '08', '09')):
            name = column[:4]+'q3'
        else:
            name = column[:4]+'q4'
        return name
    housing_data = housing_data.groupby(name_quarters, axis=1).mean().sort_index()
    
    return housing_data
print(convert_housing_data_to_quarters())

                                  2000q1         2000q2         2000q3  \
State     RegionName                                                     
Alabama   Adamsville        69033.333333   69166.666667   69800.000000   
          Alabaster        122133.333333  123066.666667  123166.666667   
          Albertville       73966.666667   72600.000000   72833.333333   
          Arab              83766.666667   81566.666667   81333.333333   
          Ardmore                    NaN            NaN            NaN   
          Axis                       NaN            NaN            NaN   
          Baileyton                  NaN            NaN            NaN   
          Bay Minette       81700.000000   78533.333333   79133.333333   
          Bayou La Batre    44066.666667   44500.000000   44266.666667   
          Bessemer                   NaN            NaN            NaN   
          Birmingham        54033.333333   54400.000000   54966.666667   
          Boaz              70866.6666

In [17]:
def quarter_before_recession():
    
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    GDP_lev = get_gdp()
    for x in range(0,GDP_lev.shape[0] - 2):
        if (GDP_lev['GDP'][x] > GDP_lev['GDP'][x+1]) & (GDP_lev['GDP'][x+1] > GDP_lev['GDP'][x+2]):
            start = GDP_lev.iloc[x-2][0]
    return start
print(quarter_before_recession())

2008q2


In [18]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    towns = get_list_of_university_towns()
    bottom = get_recession_bottom()
    quarter_before = quarter_before_recession()
    housing_data = convert_housing_data_to_quarters()
    housing_data = housing_data[[quarter_before, bottom]]
    housing_data['Price_ratio'] = housing_data[quarter_before]/housing_data[bottom]
    housing_data = housing_data.dropna()
    housing_data = housing_data.reset_index()
    housing_uni = pd.merge(towns, housing_data, how='inner', on=['State', 'RegionName'])
    housing_non_uni = housing_data[~housing_data.index.isin(housing_uni)]
    t_stat, p = ttest_ind(housing_uni['Price_ratio'], housing_non_uni['Price_ratio'])
    different = False
    if p < 0.01:
        different = True
    if t_stat < 0:
        better = 'university town'
    else:
        better = 'non-university town'
    return (different, p, better)
print(run_ttest())

(True, 0.0033861978483636051, 'university town')
